In [13]:
# GmailのAPIに必要なモジュールのインストール
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [1]:
# モジュールのインポート
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

In [2]:
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

In [3]:
def create_message(sender, to, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    return {'raw': base64.urlsafe_b64encode(message.as_string())}

In [4]:
def send_message(service, user_id, message):
    try:
        message = (service.users().messages().send(userId=user_id, body=message).execute())
        print('Message Id: %s' % message['id'])
        return message
    except errors.HttpError as e:
        print('An error occurred: %s' % error)

In [5]:
def main():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'api_key/gmail_cred.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('gmail', 'v1', credentials=creds)
    
    # メッセージの内容
    sender = 'nakashi120@gmail.com'
    to = 'nakashi120@gmail.com'
    subject = 'Test mail'
    message_text = 'This is test e-mail by Gmail API'
    
    # メッセージの作成
    message = create_message(sender, to, subject, message_text)
    send_message(service, 'me', message)

In [ ]:
main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=297053042813-6ddtokl9rf5lim4tv0l3pgidltfu29i7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A40785%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=WshPMcFTfKnDzduclzVVQNAnavyS3f&access_type=offline
